In [29]:
## brail letter #
import time
import PIL

import torch
from ultralyticsplus import YOLO, render_result

import json
import numpy as np
import torch


def convert_to_braille_unicode(str_input: str, path: str = "./braille-map.json") -> str:
    with open(path, "r") as fl:
        data = json.load(fl)

    if str_input in data.keys():
        str_output = data[str_input]
    return str_output


def parse_xywh_and_class(boxes: torch.Tensor) -> list:
    """
    boxes input tensor
        boxes (torch.Tensor) or (numpy.ndarray): A tensor or numpy array containing the detection boxes,
            with shape (num_boxes, 6).
        orig_shape (torch.Tensor) or (numpy.ndarray): Original image size, in the format (height, width).

    Properties:
        xyxy (torch.Tensor) or (numpy.ndarray): The boxes in xyxy format.
        conf (torch.Tensor) or (numpy.ndarray): The confidence values of the boxes.
        cls (torch.Tensor) or (numpy.ndarray): The class values of the boxes.
        xywh (torch.Tensor) or (numpy.ndarray): The boxes in xywh format.
        xyxyn (torch.Tensor) or (numpy.ndarray): The boxes in xyxy format normalized by original image size.
        xywhn (torch.Tensor) or (numpy.ndarray): The boxes in xywh format normalized by original image size.
    """

    # copy values from troublesome "boxes" object to numpy array
    new_boxes = np.zeros(boxes.shape)
    new_boxes[:, :4] = boxes.xywh.numpy()  # first 4 channels are xywh
    new_boxes[:, 4] = boxes.conf.numpy()  # 5th channel is confidence
    new_boxes[:, 5] = boxes.cls.numpy()  # 6th channel is class which is last channel

    # sort according to y coordinate
    new_boxes = new_boxes[new_boxes[:, 1].argsort()]

    # find threshold index to break the line
    y_threshold = np.mean(new_boxes[:, 3]) // 2
    boxes_diff = np.diff(new_boxes[:, 1])
    threshold_index = np.where(boxes_diff > y_threshold)[0]

    # cluster according to threshold_index
    boxes_clustered = np.split(new_boxes, threshold_index + 1)
    boxes_return = []
    for cluster in boxes_clustered:
        # sort according to x coordinate
        cluster = cluster[cluster[:, 0].argsort()]
        boxes_return.append(cluster)

    return boxes_return

def load_model(model_path):
    """load model from path"""
    model = YOLO(model_path)
    return model


def load_image(image_path):
    """load image from path"""
    image = PIL.Image.open(image_path)
    return image

model_path = "./yolov8m.pt"

try:
    model = load_model(model_path)
    model.overrides["conf"] = 0.10  # NMS confidence threshold
    model.overrides["iou"] = 0.20  # NMS IoU threshold
    model.overrides["agnostic_nms"] = False  # NMS class-agnostic
    model.overrides["max_det"] = 1000  # maximum number of detections per image

    image = load_image("./alpha-numeric.jpeg")

    with torch.no_grad():
        res = model.predict(image, save=False, save_txt=False, exist_ok=True, conf=0.10)
        boxes = res[0].boxes  # first image
        res_plotted = res[0].plot()[:, :, ::-1]
        list_boxes = parse_xywh_and_class(boxes)
        str_predict = []
        for box_line in list_boxes:
            box_classes = box_line[:, -1]
            for each_class in box_classes:
                str_predict.append(model.names[int(each_class)])
        print(str_predict)
except Exception as ex:
    print(ex)
    # st.write(f"Unable to load model. Check the specified path: {model_path}")



Ultralytics YOLOv8.0.43 🚀 Python-3.10.12 torch-2.1.1+cu121 CPU
Model summary (fused): 218 layers, 25891088 parameters, 0 gradients, 79.7 GFLOPs

0: 640x640 2 001111s, 2 010100s, 2 010110s, 1 011100, 1 011110, 2 100000s, 4 100010s, 2 100100s, 4 100110s, 1 101001, 1 101010, 1 101011, 1 101100, 1 101101, 1 101110, 1 101111, 3 110000s, 3 110010s, 2 110100s, 2 110110s, 2 111000s, 1 111001, 2 111010s, 1 111100, 1 111110, 404.5ms
Speed: 0.7ms preprocess, 404.5ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)


['100000', '110000', '100100', '110100', '100110', '100010', '100110', '110100', '110110', '110010', '010100', '100010', '110010', '111000', '111000', '101100', '101110', '101010', '111100', '111110', '111010', '011100', '011110', '101001', '001111', '111010', '101101', '101111', '101011', '001111', '111001', '110010', '010110', '100000', '110000', '100100', '100110', '100010', '100110', '110110', '010110', '010100', '100010', '110000']
